In [1]:
from transformers import pipeline, AutoTokenizer
classifier = pipeline("sentiment-analysis")
result = classifier("I love how simple this API is!")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print(result)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9997503161430359}]


In [2]:
import pandas as pd
import os
from tqdm import tqdm

In [6]:
input_folder = '/Users/samxie/Research/HEC/Reddit Sentiment 0417/Data_Before2'
output_folder = '/Users/samxie/Research/HEC/Reddit Sentiment 0417/Data_After'


MAX_SEQ_LEN = 512


for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):
        
        file_path = os.path.join(input_folder, file_name)
        df = pd.read_csv(file_path, lineterminator='\n')
        
        for i in tqdm(range(len(df)), desc=f"Processing {file_name}", ncols=100):
            text = str(df.loc[i, 'body'])  
            
            if text.strip() == "":
                continue
            
            encoding = tokenizer(text, truncation=True, padding=False, max_length=MAX_SEQ_LEN)
            truncated_text = tokenizer.decode(encoding['input_ids'], skip_special_tokens=True)
            
            sentiment = classifier(truncated_text)[0]
            df.loc[i, 'Sentiment_Label'] = 1 if sentiment['label'] == 'POSITIVE' else -1
            df.loc[i, 'Sentiment_Score'] = round(sentiment['score'], 4)
            df.loc[i, 'Sentiment_Score_Final'] = df.loc[i, 'Sentiment_Label'] * df.loc[i, 'Sentiment_Score']
        
        new_file_name = file_name.replace('_after_toxic', '')
        new_file_name = new_file_name.replace('.csv', '_done.csv')
        new_file_path = os.path.join(output_folder, new_file_name)
        
        df.to_csv(new_file_path, index=False)
        
        print(f"Processed file saved to {new_file_path}")

Processing BethesdaSoftworks_comments_deleted_extracted_after_toxic.csv: 100%|█| 5066/5066 [02:10<00


Processed file saved to /Users/samxie/Research/HEC/Reddit Sentiment 0417/Data_After/BethesdaSoftworks_comments_deleted_extracted_done.csv


Processing Volkswagen_comments_deleted_extracted_after_toxic.csv: 100%|█| 24344/24344 [11:14<00:00, 


Processed file saved to /Users/samxie/Research/HEC/Reddit Sentiment 0417/Data_After/Volkswagen_comments_deleted_extracted_done.csv


Processing gopro_comments_removed_extracted_after_toxic.csv: 100%|█| 2618/2618 [01:12<00:00, 36.16it


...

Processed file saved to /Users/samxie/Research/HEC/Reddit Sentiment 0417/Data_After/gopro_comments_deleted_extracted_done.csv


Processing intel_comments_deleted_extracted_after_toxic.csv: 100%|█| 37156/37156 [18:28<00:00, 33.52


Processed file saved to /Users/samxie/Research/HEC/Reddit Sentiment 0417/Data_After/intel_comments_deleted_extracted_done.csv


Processing ASUS_comments_removed_extracted_after_toxic.csv: 100%|█| 2128/2128 [00:59<00:00, 35.65it/

Processed file saved to /Users/samxie/Research/HEC/Reddit Sentiment 0417/Data_After/ASUS_comments_removed_extracted_done.csv
